<a href="https://colab.research.google.com/github/mahesh-tippanu/liberay_bPML/blob/feature%2Ffeature-engineering/liberay_bPML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Preprocessing

Load the dataset using pandas from an Excel file.
Handle missing values by filling them with median (for numerical columns like SNIP and SJR) and mode (for categorical columns like Publisher).
Drop columns with excessive missing values (Print ISSN and E-ISSN).
Handle duplicate entries using drop_duplicates().

In [1]:
import pandas as pd

file_path = '/content/drive/MyDrive/Machine Learning Project/Ist Quratile journals Scopus (1).xlsx'
df = pd.read_excel(file_path, sheet_name='Sheet1')

# Handle missing values
df['SNIP'].fillna(df['SNIP'].median(), inplace=True)
df['SJR'].fillna(df['SJR'].median(), inplace=True)
df['Publisher'].fillna(df['Publisher'].mode()[0], inplace=True)
df['Main Publisher'].fillna(df['Main Publisher'].mode()[0], inplace=True)

# Drop columns with excessive missing values
df_cleaned = df.drop(columns=['Print ISSN', 'E-ISSN'])

# Handle duplicate entries
df_cleaned.drop_duplicates(inplace=True)

# Save the cleaned data for the next step
df_cleaned.to_excel('cleaned_journal_data.xlsx', index=False)


<ipython-input-1-5804d5e88c0c>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['SNIP'].fillna(df['SNIP'].median(), inplace=True)
<ipython-input-1-5804d5e88c0c>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

Feature-engineering Aggregate the dataset by grouping duplicates based on Scopus Source ID and RANK, taking the mean of the Top 10% (CiteScore Percentile) feature. Pivot the data to create a user-item matrix, where rows represent journals (based on Scopus Source ID), columns represent RANK, and values represent Top 10% (CiteScore Percentile)

In [1]:
#pip install numpy pandas scikit-learn seaborn


Accessing Files: Files in your Drive are accessible under /content/drive/MyDrive/.
Adjust the path to match the folder and file structure in your Google Drive.


In [2]:
import pandas as pd

file_path = '/content/drive/MyDrive/Machine Learning Project/Ist Quratile journals Scopus (1).xlsx'
df = pd.read_excel(file_path, sheet_name='Sheet1')
print(df.head())


   Scopus Source ID                                     Title  Citation Count  \
0             12091         Atmospheric Chemistry and Physics           34849   
1             12459  Journal of the National Cancer Institute           11530   
2             12459  Journal of the National Cancer Institute           11530   
3             13877   European Journal of Mechanics, A/Solids            6294   
4             13877   European Journal of Mechanics, A/Solids            6294   

   Scholarly Output  Percent Cited  CiteScore   SNIP    SJR  \
0              3251             85       10.7  1.291  2.138   
1               679             88       17.0  2.556  4.986   
2               679             88       17.0  2.556  4.986   
3               899             83        7.0  1.447  0.993   
4               899             83        7.0  1.447  0.993   

   Scopus ASJC Code (Sub-subject Area)      Scopus Sub-Subject Area  ...  \
0                                 1902          Atmospheri